In [1]:
import pandas as pd
import numpy as np
import math
import scipy.stats as stats
import random as ran

In [2]:
activeThreshold = 2 
runSayisi = 3 # initial run sayısı
d2 = 1.693
RmThres = 1.07
#coef = np.array([25,112,0.4,1111,0.01,48,3])
interc = 0
run = 0
totalRun = 0


In [3]:
f3 = pd.read_excel('1.xlsx', index_col=0, sheet_name='3Factor')
f3Err = pd.read_excel('1.xlsx', index_col=0, sheet_name='3Error')
f9 = pd.read_excel('1.xlsx', index_col=0, sheet_name='9Factor')
f9Err = pd.read_excel('1.xlsx', index_col=0, sheet_name='9Error')

In [4]:
    # settings
    factor = 9
    nfactor = 3
    error = 'ErrorLow'
    factorExcel = f9
    errorExcel =  f9Err
    '''
    np.random.normal(0, 2)
    np.random.normal(0, 8)
    '''

'\nnp.random.normal(0, 2)\nnp.random.normal(0, 8)\n'

## dikkat edilmesi gerekenler

shainin VS metodunda faktörlerin önem sırasına göre sıralandığı varsayılır. Faktörler önem sırasına göre sıralanmalıdır.


In [5]:
def fillInteractions(X):
        # Fill Interaction Terms
    X = pd.DataFrame(X)
    shape = X.shape
    num_measurement = shape[0]
    num_factors = shape[1]
    num_interactions = num_factors*(num_factors-1)/2
    interaction_counter1 = 0
    interaction_counter2 = 1
    interaction_counter3 = num_factors - 1
    title_counter = num_factors
    for i in range(num_factors):
        for j in range(interaction_counter3):
            add_column = X[interaction_counter1] * X[interaction_counter2 + j]
            X[str(title_counter)] = add_column
            title_counter = title_counter + 1
        interaction_counter1 = interaction_counter1 + 1
        interaction_counter2 = interaction_counter2 + 1
        interaction_counter3 = interaction_counter3 - 1

    # Fill Quadratic Terms
    title_counter = num_factors + num_interactions
    for i in range(num_factors):
        add_column = X[i]*X[i]
        X[str(title_counter)] = add_column
        title_counter = title_counter + 1
    return X

In [6]:
def runModel(Xrr): 
    y = np.zeros((Xrr.shape[0],1))
    X1 = pd.DataFrame(Xrr)
    X = fillInteractions(X1)
    Xmat = X
    a = Xrr.shape[0]
    b = Xrr.shape[1]
    '''
    facts = np.ones((1,1))
    for i in Xrr:
        i = i.astype(int)
        facts1 = ",".join(map(str, i))
        facts = np.vstack([facts, facts1]) 
    facts = np.delete(facts, 0,0)
    a = pd.DataFrame(facts)
    X['facts'] = facts
    print(facts)
    '''

    for index1, row1 in X1.iterrows():
        for index2, row2 in factorExcel.iterrows():
            if nfactor == row2['f']:
                coef = row2[1:]
                xx1 = np.array(row1[0:])
                xx2 = np.sum(np.multiply(xx1, coef))
                y[index1,0] = xx2 + np.random.normal(0, 2)
    global run
    run = run + y.shape[1]
    '''
    X.to_excel('9FF.xlsx', engine='xlsxwriter')
    '''
    return y

In [7]:
def stage1X(factor):
    Xbest = np.ones((runSayisi,factor))
    Xworst = -1*np.ones((runSayisi,factor))
    return Xbest, Xworst

In [8]:
def computeR(ybest, yworst):
    print('ybest, yworst')
    print(ybest,'- ',  yworst)
    Mb = np.median(ybest)
    Mw = np.median(yworst)
    Rb = np.max(ybest) - np.min(ybest)
    Rw = np.max(yworst) - np.min(yworst)
    Rm = (Mb-Mw)/ ((Rb+Rw)/2)
    print('Rm:', Rm)
    return Rm

In [9]:
def step1(factor):
    Xbest, Xworst = stage1X(factor)
    print('Xbest')
    print(Xbest)
    ybest = runModel(Xbest)
    print('ybest')
    print(ybest)
    yworst = runModel(Xworst)
    Rm = computeR(ybest, yworst)
    return Rm, ybest, yworst

In [10]:
# CI calculation with t test (df 4)
def calculateCI(ybest, yworst):
    Mb = np.median(ybest)
    Mw = np.median(yworst)
    Rb = np.max(ybest) - np.min(ybest)
    Rw = np.max(yworst) - np.min(yworst)
    Ravg= ((Rb+Rw)/2)
    interval = (2.776*Ravg)/d2
    CIb = np.array([Mb-interval, Mb+interval])
    CIw = np.array([Mw-interval,Mw+interval])
    print('CIb : ',CIb)
    print('CIw : ',CIw)
    return CIb, CIw

In [11]:
def CIcontrol(y, CIb, CIw):
    print('----------------------------------------------------------')
    print(y.shape, CIb.shape, CIw.shape)
    if ((CIb[0] < y[0][0]) and (y[0][0] < CIb[1])) and ((CIw[0] < y[0][1]) and (y[0][1] < CIw[1])):
        return True
    else:
        return False

In [12]:
def swapping(i,CIb, CIw):
    X1 = np.ones((1,factor))
    X1[0,i] = -1
    X = X1
    X = np.vstack([X, -1*X1])  
    y = runModel(X)
    return CIcontrol(y, CIb, CIw)

In [13]:
def capping(actives,CIb, CIw):
    if actives.sum() >= activeThreshold:
        activesX = actives*2 - 1
        activesX = np.vstack([activesX, -1*activesX])
        y = runModel(activesX)
        return CIcontrol(y, CIb, CIw)
    else:
        return False

In [14]:
def sonucYazdir(p, beta, sigmaSqP, RoP):
    print('                 Results                 ')
    print('---------------------------------------- ')
    print('Number of active factors: ', p)
    print('Pairwise correlation:  ', RoP)
    print('Variance of estimated main effects: ', sigmaSqP)
    print('Coefficietns:')
    print(coef)
    print('Beta matrix: ')
    print(beta)
    print('Total run number: ')
    print(totalRun)

In [15]:
# Xp bulma
def findXp(p):
    X1 = np.ones((1,p))
    X = X1
    X = np.vstack([X, -1*X1])  
    for i in range(p):
        X1 = np.ones((1,p))
        X1[0][i] = -1
        X = np.vstack([X, X1])
        X = np.vstack([X, -1*X1])
    return X

In [16]:
# B main bulma
def betaMatris(Xp, y):
    XpT = np.transpose(Xp)
    tmp1 = np.linalg.inv(XpT.dot(Xp))
    tmp2 = tmp1.dot(XpT)
    beta = tmp2.dot(np.transpose(y))
    return beta

In [17]:
# sigmaP :variance of estimated main effects when there are p active factors
def calculateVariance(p, sigmaSq):
    return ((p*p-4*p+7)*sigmaSq)/(8*(p*p-3*p+4))

In [18]:
# roP : pairwise correlation of estimated main effects when there are p active factors
def calculateRoP(p):
    return -(p-3)/(p*p-4*p+7)

In [19]:
def step2(ybest, yworst):
    # CI hesaplama
    CIb, CIw = calculateCI(ybest, yworst)

    # aktif faktörleri tutan array
    actives = np.zeros((1,factor))
    i=0

    while i<factor:
        if not swapping(i, CIb, CIw):
            actives[0][i]=1
            print(i+1,'. faktör aktiftir.')
            if capping(actives,CIb, CIw):
                print('Algoritma tamamlandı')
                break
            else:
                i = i+1
        else:
            i = i+1
    return actives

In [20]:
def findXpAll(XpAll, actives):
    for i in actives[0]:
        if i == 0:
            XpAll = np.insert(XpAll, int(i), 0, axis = 1) 
    return XpAll

In [21]:
def findXpAll(XpAll, actives):
    for i in actives[0]:
        if i == 0:
            XpAll = np.insert(XpAll, int(i), 0, axis = 1) 
    return XpAll

In [22]:
# algoritma
Rm, ybest, yworst = step1(factor)
if  Rm > RmThres:
    print('Move on to the next stage!')
    actives = step2(ybest, yworst)
else:
    print('There is no active factor!')
print('Active factores: ')
print(actives)
# p = int(actives.sum()) # p represents # of active factors
#print('p:  ', p)
#Xp = findXp(p)
# y = runModel(findXpAll(Xp, actives))
p = factor
Xp = findXp(p)
totalRun = run
y = runModel(Xp)

beta = betaMatris(Xp, y)
sigmaSq = 1
sigmaSqP = calculateVariance(p, sigmaSq)
RoP = calculateRoP(p)
sonucYazdir(p, beta, sigmaSqP, RoP)



Xbest
[[1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1.]]
ybest
[[-47.92302709]
 [-48.60050235]
 [-50.34890826]]
ybest, yworst
[[-47.92302709]
 [-48.60050235]
 [-50.34890826]] -  [[-84.93248618]
 [-88.94677991]
 [-88.23583611]]
Rm: 12.308775591806723
Move on to the next stage!
CIb :  [-53.88045672 -43.32054798]
CIw :  [-93.51579048 -82.95588174]
----------------------------------------------------------
(2, 1) (2,) (2,)
1 . faktör aktiftir.
----------------------------------------------------------
(2, 1) (2,) (2,)
2 . faktör aktiftir.
----------------------------------------------------------
(2, 1) (2,) (2,)
----------------------------------------------------------
(2, 1) (2,) (2,)
3 . faktör aktiftir.
----------------------------------------------------------
(2, 1) (2,) (2,)


IndexError: index 1 is out of bounds for axis 0 with size 1

In [ ]:
for i in range(factor):
    print(coef[0]/beta[0])

yaapılacaklar :
    r sq
    performans measure ları